In [1]:
import pandas as pd
import matplotlib.pyplot as plt
orders = pd.read_csv('./data/Orders.csv')
returns = pd.read_csv('./data/Returns.csv')
orders_copy = orders
orders

,Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,...,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority
0,40098,CA-2014-AB10015140-41954,11/11/14,11/13/14,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,$221.98,2,0.0,$62.15,40.770,High
1,26341,IN-2014-JR162107-41675,2/5/14,2/7/14,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black","$3,709.40",9,0.1,-$288.77,923.630,Critical
2,25330,IN-2014-CR127307-41929,10/17/14,10/18/14,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID","$5,175.17",9,0.1,$919.97,915.490,Medium
3,13524,ES-2014-KM1637548-41667,1/28/14,1/30/14,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless","$2,892.51",5,0.1,-$96.54,910.160,Medium
4,47221,SG-2014-RH9495111-41948,11/5/14,11/6/14,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed","$2,832.96",8,0.0,$311.52,903.040,Critical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2015-KE1642066-42174,6/19/15,6/19/15,Same Day,KE-1642066,Katrina Edelman,Corporate,NaN,Kure,...,OFF-FA-3072,Office Supplies,Fasteners,"Advantus Thumb Tacks, 12 Pack",$65.10,5,0.0,$4.50,1.010,Medium
51286,34337,US-2014-ZD21925140-41765,5/6/14,5/10/14,Standard Class,ZD-219251408,Zuschuss Donatelli,Consumer,37421.0,Chattanooga,...,FUR-FU-4070,Furniture,Furnishings,"Eldon Image Series Desk Accessories, Burgundy",$16.72,5,0.2,$3.34,1.930,High
51287,31315,CA-2012-ZD21925140-41147,8/26/12,8/31/12,Second Class,ZD-219251404,Zuschuss Donatelli,Consumer,94109.0,San Francisco,...,OFF-AR-5321,Office Supplies,Art,Newell 341,$8.56,2,0.0,$2.48,1.580,High
51288,9596,MX-2013-RB1979518-41322,2/17/13,2/21/13,Standard Class,RB-1979518,Ross Baird,Home Office,NaN,Valinhos,...,OFF-BI-2919,Office Supplies,Binders,"Acco Index Tab, Economy",$13.44,2,0.0,$2.40,1.003,Medium


In [98]:
orders.columns
orders_copy.dtypes

Row.ID                     int64
Order.ID                  object
Order.Date        datetime64[ns]
Ship.Date                 object
Ship.Mode                 object
Customer.ID               object
Customer.Name             object
Segment                   object
Postal.Code              float64
City                      object
State                     object
Country                   object
Region                    object
Market                    object
Product.ID                object
Category                  object
Sub.Category              object
Product.Name              object
Sales                    float64
Quantity                   int64
Discount                 float64
Profit                   float64
Shipping.Cost            float64
Order.Priority            object
dtype: object

In [2]:
orders_copy['Profit'] = orders_copy['Profit'].str.replace('$', '').str.replace(',', '').astype('float')
orders_copy['Sales'] = orders_copy['Sales'].str.replace('$', '').str.replace(',', '').astype('float')

In [149]:
orders_copy['Order.Date'] = pd.to_datetime(orders_copy['Order.Date'])
seasonal = orders_copy[['Order.Date', 'Quantity', 'Category']]
seasonal = seasonal.set_index('Order.Date')
# seasonal = seasonal.groupby(['Order.Date', 'Category']).agg({'Quantity': 'sum'})
seasonal

,Quantity,Category
Order.Date,,
2014-11-11,2,Technology
2014-02-05,9,Furniture
2014-10-17,9,Technology
2014-01-28,5,Technology
2014-11-05,8,Technology
...,...,...
2015-06-19,5,Office Supplies
2014-05-06,5,Furniture
2012-08-26,2,Office Supplies


In [34]:
returns_copy = returns.rename(columns={'Order ID': 'Order.ID'})
combined = orders_copy.merge(returns_copy, on='Order.ID', how='left')
returned_count = combined.groupby(['Customer.Name']).agg({'Returned': 'count'}).sort_values(by=['Returned'], ascending=False)
returned_count.loc[returned_count['Returned'] > 5]
returned_region = combined.groupby(['Region_y']).agg({'Returned': 'count'}).sort_values(by=['Returned'], ascending=False)
returned_categories = combined.groupby(['Category']).agg({'Returned': 'count'}).sort_values(by=['Returned'], ascending=False)
returned_categories


,Returned
Category,
Office Supplies,1348
Technology,445
Furniture,427


In [35]:
combined_copy = combined
combined_copy
combined_copy['Returned'].loc[combined_copy['Returned'] == 'Yes']
combined_copy['Returned'] = combined_copy['Returned'].apply(lambda x: 1 if x == 'Yes' else 0)
combined_copy['Order.Date'] = pd.to_datetime(combined_copy['Order.Date'])
combined_copy['Ship.Date'] = pd.to_datetime(combined_copy    ['Ship.Date'])
combined_copy['Process.Time'] = combined_copy['Ship.Date'] - combined_copy['Order.Date']

In [130]:
returned_ids = combined_copy.groupby(['Product.ID']).agg({'Returned':'count'}).sort_values(by=['Returned'], ascending=False)
combined_copy.loc[combined_copy['Product.ID'].isin(returned_ids.index.values)]
returned_ids['Product.ID'] = returned_ids.index
returned_ids.rename({'Returned': 'Returned.Count'}, axis=1, inplace=True)
returned_ids = returned_ids.reset_index(drop=True)
final = combined_copy.merge(returned_ids, on='Product.ID')

,Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,...,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority,Returned,Region_y,Process.Time,Returned.Count


0        0
1        0
2        0
3        0
4        0
        ..
51285    0
51286    0
51287    0
51288    0
51289    0
Name: Returned, Length: 49070, dtype: int64